In [3]:
import numpy as np
import pickle
import random

In [4]:
filenames = pickle.load(open('data/filenames-caltech101.pickle', 'rb'))
feature_list = pickle.load(open('data/features-caltech101-mobilenet.pickle',
                                'rb'))

In [5]:
num_images = len(filenames)
num_features_per_image = len(feature_list[0])

PCA

In [6]:
num_feature_dimensions = 100
num_feature_dimensions = min(num_images, num_feature_dimensions,
                             len(feature_list[0]))

In [7]:
from sklearn.decomposition import PCA

In [8]:
pca = PCA(n_components=num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list)
feature_list_compressed = feature_list_compressed.tolist()

In [10]:
!pip install nmslib

     |████████████████████████████████| 899 kB 1.6 MB/s eta 0:00:01


In [11]:
import nmslib


In [13]:
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(feature_list_compressed)
index.createIndex({'post': 2}, print_progress=True)

nmslib on one image

In [15]:
random_image_index = random.randint(0, num_images)

In [16]:
# Query for the nearest neighbors of the first datapoint
%timeit index.knnQuery(feature_list_compressed[random_image_index], k=5)
ids, distances = index.knnQuery(feature_list_compressed[random_image_index],
                                k=5)

29.1 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


on total dataset

In [17]:
# Get all nearest neighbors for all the datapoint
%timeit index.knnQueryBatch(feature_list_compressed, k=5, num_threads=16)
neighbors = index.knnQueryBatch(feature_list_compressed, k=5, num_threads=16)

114 ms ± 4.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
